In [1]:
#Imports
import collections
import re

from conf.config import Config
from pathlib import Path
import json
import pandas as pd
from mo_sql_parsing import parse
from utils.SQLParser import SQLParser

1. Parse train_spider.json to dict

In [10]:
json_path = Path(Config.spider_folder +'/train_spider.json')
spider_queries_df = pd.read_json(json_path)
spider_queries_df = spider_queries_df.drop(columns =['sql'])
spider_queries_df

,db_id,query,query_toks,query_toks_no_value,question,question_toks
0,department_management,SELECT count(*) FROM head WHERE age > 56,"[SELECT, count, (, *, ), FROM, head, WHERE, ag...","[select, count, (, *, ), from, head, where, ag...",How many heads of the departments are older th...,"[How, many, heads, of, the, departments, are, ..."
1,department_management,"SELECT name , born_state , age FROM head ORD...","[SELECT, name, ,, born_state, ,, age, FROM, he...","[select, name, ,, born_state, ,, age, from, he...","List the name, born state and age of the heads...","[List, the, name, ,, born, state, and, age, of..."
2,department_management,"SELECT creation , name , budget_in_billions ...","[SELECT, creation, ,, name, ,, budget_in_billi...","[select, creation, ,, name, ,, budget_in_billi...","List the creation year, name and budget of eac...","[List, the, creation, year, ,, name, and, budg..."
3,department_management,"SELECT max(budget_in_billions) , min(budget_i...","[SELECT, max, (, budget_in_billions, ), ,, min...","[select, max, (, budget_in_billions, ), ,, min...",What are the maximum and minimum budget of the...,"[What, are, the, maximum, and, minimum, budget..."
4,department_management,SELECT avg(num_employees) FROM department WHER...,"[SELECT, avg, (, num_employees, ), FROM, depar...","[select, avg, (, num_employees, ), from, depar...",What is the average number of employees of the...,"[What, is, the, average, number, of, employees..."
...,...,...,...,...,...,...
6995,culture_company,SELECT T1.company_name FROM culture_company AS...,"[SELECT, T1.company_name, FROM, culture_compan...","[select, t1, ., company_name, from, culture_co...",What are all the company names that have a boo...,"[What, are, all, the, company, names, that, ha..."
6996,culture_company,"SELECT T1.title , T3.book_title FROM movie AS...","[SELECT, T1.title, ,, T3.book_title, FROM, mov...","[select, t1, ., title, ,, t3, ., book_title, f...",Show the movie titles and book titles for all ...,"[Show, the, movie, titles, and, book, titles, ..."
6997,culture_company,"SELECT T1.title , T3.book_title FROM movie AS...","[SELECT, T1.title, ,, T3.book_title, FROM, mov...","[select, t1, ., title, ,, t3, ., book_title, f...",What are the titles of movies and books corres...,"[What, are, the, titles, of, movies, and, book..."
6998,culture_company,SELECT T2.company_name FROM movie AS T1 JOIN c...,"[SELECT, T2.company_name, FROM, movie, AS, T1,...","[select, t2, ., company_name, from, movie, as,...",Show all company names with a movie directed i...,"[Show, all, company, names, with, a, movie, di..."


2. Read the relationship json

In [11]:
relationship_path = Path(Config.spider_folder+'/consolidated_relations.json')
with open(relationship_path) as json_file:
    spider_relationship = json.load(json_file)

3. Load table in use csv file if not all tables were loaded into neo4j

In [12]:
iter_csv = pd.read_csv(Config.spider_folder + '/table_in_use.csv', iterator=True, chunksize=1000)
db_in_use_df = pd.concat([chunk[chunk['Imported'] == 1] for chunk in iter_csv])
imported_dbs = db_in_use_df['db_id'].tolist()

In [13]:
#clean up dataframe. only keep rows with db_id in the imported list
spider_queries_df = spider_queries_df[spider_queries_df['db_id'].isin(imported_dbs)].sort_values(by=['db_id'])
spider_queries_df = spider_queries_df.reset_index(drop=True)

In [14]:
#parse sql query and reformat the result before conversion
def parse_sql(query: str, db_id: str):
    parsed_query = parse(query)
    #First get table alias map to be used for replace the alias in select and where clause
    table_alias_map = {} #alias as key, table name as value
    relationships = next(relationships['db_relationships'] for relationships in spider_relationship if relationships['db_id'] == db_id)
    if 'join' not in query.lower():
        return parsed_query
    for t in parsed_query['from']:
        if 'join' not in t:
            if t['name'] is None:
                continue
            else:
                table_alias_map[t['name']] = t['value']
        else:
            table_alias_map[t['join']['name']] = t['join']['value']
    if len(table_alias_map) == 0:
        return parsed_query
    #Replace mapping table name with actual table name
    for relationship in relationships:
        table_name = list(relationship.keys())[0]
        table_relation = relationship[table_name]
        if 'mapping' not in table_relation.keys() or table_relation['mapping'] is None:
            continue
        for key in table_alias_map.keys():
            if table_alias_map[key] == table_relation['mapping']:
                table_alias_map[key] = table_relation['to']
    #print(table_alias_map)
    #replace the alias with table name then parse sql again
    for key, value in table_alias_map.items():
        query = query.replace("{}.".format(str(key)), "{}.".format(value.lower()))
        query = query.replace(".{}".format(str(key)), ".{}".format(value))
    return parse(query)

def generate_cypher(query:str, db_id:str) -> str:
    try:
        cleaned_query = parse_sql(query,db_id)
        print(cleaned_query)
        sql_parser = SQLParser()
        sql_parser.generate_cypher(cleaned_query, query, db_id)
        cypher_query = sql_parser.get_cypher()
    except Exception as e:
        cypher_query = str(e)
    return cypher_query

def convert_to_cypher(sql_query: str, db_id: str):
    if 'UNION' not in sql_query.upper() and 'UNION ALL' not in sql_query.upper():
        cypher = generate_cypher(sql_query, db_id)
    else:
        split_key_work = ''
        cypher=''
        sub_queries = ''
        if 'UNION' in sql_query.upper():
            sub_queries = re.split("union", sql_query, flags=re.IGNORECASE)
            split_key_work = 'UNION '
        elif 'UNION ALL' in sql_query.upper():
            sub_queries = re.split("union all", sql_query, flags=re.IGNORECASE)
            split_key_work = 'UNION ALL '

        for subquery in sub_queries:
            print('sub query: '+ subquery)
            if cypher == '':
                cypher += generate_cypher(subquery, db_id)
            else:
                cypher = cypher.replace(';','')
                cypher += split_key_work + generate_cypher(subquery, db_id)
    return cypher

In [20]:

sql_query = "SELECT FacID FROM Faculty EXCEPT SELECT advisor FROM Student"
db_id = 'activity_1'
multistatement_keywords = ['UNION', 'UNION ALL', 'EXCEPT', 'INTERSECT']
if not [keyword for keyword in multistatement_keywords if(keyword in sql_query.upper())]:
    cypher = generate_cypher(sql_query, db_id)
else:
    if 'UNION' in sql_query.upper():
        split_key_work = ''
        cypher=''
        split_key_work = 'UNION ALL' if 'UNION ALL' in sql_query.upper() else 'UNION'
        sub_queries = re.split(split_key_work, sql_query, flags=re.IGNORECASE)
        for subquery in sub_queries:
            print('sub query: '+ subquery)
            if cypher == '':
                cypher += generate_cypher(subquery, db_id)
            else:
                cypher = cypher.replace(';','')
                cypher += split_key_work + generate_cypher(subquery, db_id)

cypher

{'select': {'value': 'FacID'}, 'from': 'Faculty'}
['n:Faculty']
{'select': {'value': 'advisor'}, 'from': 'Student'}
['n:Student']


'MATCH (n:Faculty)  MATCH (n:Student)  RETURN  n.advisor  RETURN  n.FacID  ;'

In [16]:
#Uncomment below code for testing
#import random
#rand_row_number = random.randint(1, len(spider_queries_df))
#random_row = spider_queries_df.iloc[rand_row_number]
#random_row

In [17]:
spider_queries_df.columns.values.tolist()

['db_id',
 'query',
 'query_toks',
 'query_toks_no_value',
 'question',
 'question_toks']

In [18]:
Columns = collections.namedtuple('Columns',['Index','db_id','query','query_toks','query_toks_no_value','question','question_toks'])
spider_graph_df = spider_queries_df.copy()
valid_flags = []
for row in spider_graph_df.itertuples(index=True, name='Columns'):
    spider_graph_df.at[row.Index, 'query'] = convert_to_cypher(row.query, row.db_id)
    spider_graph_df.at[row.Index, 'query_toks'] = ''
    spider_graph_df.at[row.Index, 'query_toks_no_value'] = ''


spider_queries_df

{'select': [{'value': 'activity.activity_name'}, {'value': {'count': '*'}}], 'from': [{'value': 'Activity', 'name': 'T1'}, {'join': {'name': 'T2', 'value': 'Faculty_participates_in'}, 'on': {'eq': ['activity.actID', 'faculty_participates_in.actID']}}], 'groupby': {'value': 'activity.actID'}}
{'db_id': 'activity_1', 'db_relationships': [{'Student': {'to': 'Activity', 'relation': 'participates_in', 'mapping': 'Participates_in'}}, {'Faculty': {'to': 'Activity', 'relation': 'faculty_participates_in', 'mapping': 'Faculty_Participates_in'}}]}
['faculty:Faculty', 'activity:Activity']
{'select': [{'value': 'Fname'}, {'value': 'Lname'}, {'value': 'phone'}], 'from': 'Faculty', 'where': {'eq': ['Sex', {'literal': 'F'}]}}
['n:Faculty']
{'select': [{'value': 'Fname'}, {'value': 'Lname'}, {'value': 'phone'}], 'from': 'Faculty', 'where': {'eq': ['Sex', {'literal': 'F'}]}}
['n:Faculty']
{'select': [{'value': 'rank'}, {'value': 'Fname'}, {'value': 'Lname'}], 'from': 'Faculty'}
['n:Faculty']
{'select': 

,db_id,query,query_toks,query_toks_no_value,question,question_toks
0,activity_1,"SELECT T1.activity_name , count(*) FROM Activ...","[SELECT, T1.activity_name, ,, count, (, *, ), ...","[select, t1, ., activity_name, ,, count, (, *,...",How many faculty members participate in each a...,"[How, many, faculty, members, participate, in,..."
1,activity_1,"SELECT Fname , Lname , phone FROM Faculty WH...","[SELECT, Fname, ,, Lname, ,, phone, FROM, Facu...","[select, fname, ,, lname, ,, phone, from, facu...","What are the first name, last name, and phone ...","[What, are, the, first, name, ,, last, name, ,..."
2,activity_1,"SELECT Fname , Lname , phone FROM Faculty WH...","[SELECT, Fname, ,, Lname, ,, phone, FROM, Facu...","[select, fname, ,, lname, ,, phone, from, facu...","Show the first name, last name, and phone numb...","[Show, the, first, name, ,, last, name, ,, and..."
3,activity_1,"SELECT rank , Fname , Lname FROM Faculty","[SELECT, rank, ,, Fname, ,, Lname, FROM, Faculty]","[select, rank, ,, fname, ,, lname, from, faculty]","What are the rank, first name, and last name o...","[What, are, the, rank, ,, first, name, ,, and,..."
4,activity_1,"SELECT rank , Fname , Lname FROM Faculty","[SELECT, rank, ,, Fname, ,, Lname, FROM, Faculty]","[select, rank, ,, fname, ,, lname, from, faculty]","Show the rank, first name, and last name for a...","[Show, the, rank, ,, first, name, ,, and, last..."
...,...,...,...,...,...,...
5043,product_catalog,SELECT catalog_entry_name FROM catalog_content...,"[SELECT, catalog_entry_name, FROM, catalog_con...","[select, catalog_entry_name, from, catalog_con...",Find the name of the product that has the smal...,"[Find, the, name, of, the, product, that, has,..."
5044,product_catalog,SELECT catalog_entry_name FROM catalog_content...,"[SELECT, catalog_entry_name, FROM, catalog_con...","[select, catalog_entry_name, from, catalog_con...",Which catalog content has the highest height? ...,"[Which, catalog, content, has, the, highest, h..."
5045,product_catalog,SELECT catalog_entry_name FROM catalog_content...,"[SELECT, catalog_entry_name, FROM, catalog_con...","[select, catalog_entry_name, from, catalog_con...",What is the product with the highest height? G...,"[What, is, the, product, with, the, highest, h..."
5046,product_catalog,SELECT distinct(catalog_entry_name) FROM catal...,"[SELECT, distinct, (, catalog_entry_name, ), F...","[select, distinct, (, catalog_entry_name, ), f...",Find the names of all the catalog entries.,"[Find, the, names, of, all, the, catalog, entr..."


In [19]:
spider_graph_df

,db_id,query,query_toks,query_toks_no_value,question,question_toks
0,activity_1,MATCH (faculty:Faculty)-[:faculty_participate...,,,How many faculty members participate in each a...,"[How, many, faculty, members, participate, in,..."
1,activity_1,MATCH (n:Faculty) WHERE n.Sex = 'F' RETURN ...,,,"What are the first name, last name, and phone ...","[What, are, the, first, name, ,, last, name, ,..."
2,activity_1,MATCH (n:Faculty) WHERE n.Sex = 'F' RETURN ...,,,"Show the first name, last name, and phone numb...","[Show, the, first, name, ,, last, name, ,, and..."
3,activity_1,"MATCH (n:Faculty) RETURN n.rank, n.Fname, n.L...",,,"What are the rank, first name, and last name o...","[What, are, the, rank, ,, first, name, ,, and,..."
4,activity_1,"MATCH (n:Faculty) RETURN n.rank, n.Fname, n.L...",,,"Show the rank, first name, and last name for a...","[Show, the, rank, ,, first, name, ,, and, last..."
...,...,...,...,...,...,...
5043,product_catalog,MATCH (n:Catalog_Contents) RETURN n.catalog_...,,,Find the name of the product that has the smal...,"[Find, the, name, of, the, product, that, has,..."
5044,product_catalog,MATCH (n:Catalog_Contents) RETURN n.catalog_...,,,Which catalog content has the highest height? ...,"[Which, catalog, content, has, the, highest, h..."
5045,product_catalog,MATCH (n:Catalog_Contents) RETURN n.catalog_...,,,What is the product with the highest height? G...,"[What, is, the, product, with, the, highest, h..."
5046,product_catalog,MATCH (n:Catalog_Contents) RETURN DISTINCT n...,,,Find the names of all the catalog entries.,"[Find, the, names, of, all, the, catalog, entr..."
